In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 11.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/My Drive/cs182_final_project/cs182-nlp (master)"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import sys
sys.path.append(root_folder)

import json
import torch

In [ ]:
# result = [json.loads(jline) for jline in response.read().splitlines()]

json_file_name = "starter/yelp_review_training_dataset.jsonl"

if False:
  print("Using Sample Eval jsonl smaller file")
  json_file_name = "starter/sample_eval.jsonl"

yelp_ratings = []

with open(root_folder + "/" + json_file_name, "r") as f:
  for line in f:
    yelp_ratings.append(json.loads(line))

print(yelp_ratings)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'xlnet-base-cased')


Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_master
